In [23]:
! python -m pip install ahpy
! pip install PrettyTable

In [24]:
import numpy as np
import ahpy
import prettytable

In [3]:
cov_users = 'covered users'
impl_speed = 'implementation speed'
conv_users = 'convenience for users'
impl_cost = 'implementation cost'

In [4]:
android = 'Publish android app'
web_version = 'Improve web version for smartphones'
extended_old_sup = 'Extended support for old versions'
limited_old_sup = 'Limited support for old versions'

In [5]:
criteria_comparisons = {
    (cov_users, impl_speed): 1,
    (cov_users, conv_users): 5,
    (cov_users, impl_cost): 5,
    (impl_speed, conv_users): 3,
    (impl_speed, impl_cost): 7,
    (conv_users, impl_cost): 1,
}
print(criteria_comparisons)

{('covered users', 'implementation speed'): 1, ('covered users', 'convenience for users'): 5, ('covered users', 'implementation cost'): 5, ('implementation speed', 'convenience for users'): 3, ('implementation speed', 'implementation cost'): 7, ('convenience for users', 'implementation cost'): 1}


In [6]:
# for criteria of covered users
crit_cov_users = {
    (android, web_version): 1/5,
    (android, extended_old_sup) : 1/3,
    (android, limited_old_sup) : 1/3,
    (web_version, extended_old_sup) : 5,
    (web_version, limited_old_sup) : 5,
    (extended_old_sup, limited_old_sup): 1,
}

In [7]:
# for criteria of implementation speed
crit_impl_speed = {
    (android, web_version): 7, 
    (android, extended_old_sup) : 5,
    (android, limited_old_sup) : 3,
    (web_version, extended_old_sup) : 1/3,
    (web_version, limited_old_sup) : 1/3,
    (extended_old_sup, limited_old_sup) : 1/3
}

In [8]:
# for criteria of convenience for users
crit_conv_users = { 
    (android, web_version): 7,
    (android, extended_old_sup) : 5,
    (android, limited_old_sup) : 5,
    (web_version, extended_old_sup): 1/3,
    (web_version, limited_old_sup) : 1/3,
    (extended_old_sup, limited_old_sup) : 2,
}

In [9]:
# for criteria of implementation cost
crit_impl_cost = {
    (android, web_version): 7,
    (android, extended_old_sup): 9,
    (android, limited_old_sup): 5,
    (web_version, extended_old_sup) : 5,
    (web_version, limited_old_sup) : 2,
    (extended_old_sup, limited_old_sup) : 1/3
}

In [10]:
# creating comparisons
comp_cov_users = ahpy.Compare(cov_users, crit_cov_users, precision=3, random_index='saaty')
comp_impl_speed = ahpy.Compare(impl_speed, crit_impl_speed, precision=3, random_index='saaty')
comp_conv_users = ahpy.Compare(conv_users, crit_conv_users, precision=3, random_index='saaty')
comp_impl_cost = ahpy.Compare(impl_cost, crit_impl_cost, precision=3, random_index='saaty')

comp_criteria = ahpy.Compare('Criteria', criteria_comparisons, precision=3, random_index="saaty")
comp_criteria.add_children([comp_cov_users, comp_impl_speed, comp_conv_users, comp_impl_cost])

In [11]:
comp_criteria.target_weights

{'Publish android app': 0.377,
 'Improve web version for smartphones': 0.301,
 'Limited support for old versions': 0.185,
 'Extended support for old versions': 0.137}

In [13]:
report = comp_criteria.report(show=True)

{
    "name": "Criteria",
    "global_weight": 1.0,
    "local_weight": 1.0,
    "target_weights": {
        "Publish android app": 0.377,
        "Improve web version for smartphones": 0.301,
        "Limited support for old versions": 0.185,
        "Extended support for old versions": 0.137
    },
    "elements": {
        "global_weights": {
            "covered users": 0.417,
            "implementation speed": 0.408,
            "convenience for users": 0.097,
            "implementation cost": 0.078
        },
        "local_weights": {
            "covered users": 0.417,
            "implementation speed": 0.408,
            "convenience for users": 0.097,
            "implementation cost": 0.078
        },
        "consistency_ratio": 0.026
    }
}


In [28]:
pret = prettytable.PrettyTable()

pret.field_names = ["comparison matrix", "consistency ratio (%)"]

def format(cr):
    return "{cr: .1f} %".format(cr=cr*100)

pret.add_row(["criteria", format(cr=comp_criteria.consistency_ratio)])

for i in [comp_cov_users, comp_impl_speed, comp_conv_users, comp_impl_cost]:
    assert(i.consistency_ratio < 0.1)
    pret.add_row([i.name, format(cr=i.consistency_ratio)])
print(pret)

+-----------------------+-----------------------+
|   comparison matrix   | consistency ratio (%) |
+-----------------------+-----------------------+
|        criteria       |          2.6 %        |
|     covered users     |          5.8 %        |
|  implementation speed |          5.2 %        |
| convenience for users |          5.0 %        |
|  implementation cost  |          7.7 %        |
+-----------------------+-----------------------+
